In [1]:
import tianshou as ts
import torch
import gym

env = gym.make('Ant-v3')

In [2]:
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
max_action = env.action_space.high[0]

In [4]:
train_envs = ts.env.SubprocVectorEnv([lambda: gym.make('Ant-v3') for _ in range(20)], 
                                     norm_obs=True)
test_envs = ts.env.SubprocVectorEnv(
        [lambda: gym.make('Ant-v3') for _ in range(60)],
        norm_obs=True,
        obs_rms=train_envs.obs_rms,
        update_obs_rms=False
    )

In [6]:
policy = ts.policy.DQNPolicy(net, optim, discount_factor = 0.9, estimation_step = 3, target_update_freq = 320)

In [7]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise = True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise = True)

In [8]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold)
print(f'Finished training! Use {result["duration"]}')

ValueError: Action dimension mismatch